In [4]:
import open3d as o3d
import numpy as np

file = '../../../KITTI_partial_3s/velodyne_rov/181432.bin'

# pcd = o3d.io.read_point_cloud(file)
# o3d.visualization.draw_geometries([pcd])

In [5]:
# # HDL-32E
# n_scan = 32 # vertical channels
# horizon_scan = 1800 # horizontal channels
# ang_top = 10.67 # max vertical angle
# ang_bottom = -30.67 # min vertical angle
# ground_scan_ind = 20

# # VLP-32C
# n_scan = 32 # vertical channels
# horizon_scan = 1800 # horizontal channels
# ang_top = 15 # max vertical angle
# ang_bottom = -25 # min vertical angle
# ground_scan_ind = 20

# HDL-64E
n_scan = 64 # vertical channels
horizon_scan = 1800 # horizontal channels
ang_top = 2 # max vertical angle
ang_bottom = -24.9 # min vertical angle
ground_scan_ind = 40

ang_res_x = 360/horizon_scan
ang_res_y = (ang_top-ang_bottom) / (n_scan - 1)

sensor_mount_angle = 0
sensor_minimum_range = 1

# pcd = o3d.io.read_point_cloud(file)
# coords = np.asarray(pcd.points)

coords = np.fromfile(file, dtype=np.float32).reshape((-1, 4))
coords = coords[:, 0:3]

# Reorder points
reordered_coords = np.zeros(shape=[horizon_scan * n_scan, 4])
for [x, y, z] in coords:
    if np.linalg.norm([x, y, z]) < sensor_minimum_range:
        continue

    vertical_angle = np.arctan2(z, np.linalg.norm([x, y])) * 180.0 / np.pi

    if vertical_angle < ang_bottom or vertical_angle > ang_top:
        continue

    vert_id = np.round((vertical_angle - ang_bottom) / ang_res_y)

    horizon_angle = np.arctan2(y, x) * 180.0 / np.pi

    if horizon_angle < 0:
        horizon_angle += 360

    horizon_id = np.round(horizon_angle / ang_res_x)

    index = int(horizon_id + vert_id * horizon_scan)

    reordered_coords[index, :3] = [x, y, z]

# Ground removal
for i in range(reordered_coords.shape[0]):
    lower_id = i
    upper_id = i + horizon_scan

    if upper_id >= reordered_coords.shape[0]:
        continue

    diffX = reordered_coords[upper_id, 0] - reordered_coords[lower_id, 0]
    diffY = reordered_coords[upper_id, 1] - reordered_coords[lower_id, 1]
    diffZ = reordered_coords[upper_id, 2] - reordered_coords[lower_id, 2]

    angle = np.arctan2(diffZ, np.linalg.norm([diffX, diffY])) * 180.0 / np.pi

    if np.abs(angle - sensor_minimum_range) <= 10:
        reordered_coords[i, 3] = 1

reordered_coords = reordered_coords[np.logical_not(np.logical_and(reordered_coords[:, 0] == 0, reordered_coords[:, 1] == 0, reordered_coords[:, 2] == 0))]
segmented_coords = reordered_coords[:, :3][reordered_coords[:, -1] != 1]


In [6]:
pcd_seg = o3d.geometry.PointCloud()
pcd_seg.points = o3d.utility.Vector3dVector(segmented_coords)
o3d.visualization.draw_geometries([pcd_seg])

In [8]:
np.savetxt('/mnt/F/Uni GDrive/UniMelb/PhD/scripts/code_lidar/demo/550020.0497.csv', segmented_coords, delimiter=',')